In [1]:
import numpy as np

## 1. Regression coefficients

In [2]:
#Linear regression coefficients for the Capacity Index model (Table 3 in paper Van Dorsser et al (2020))
CI_coefs = dict({"intercept": 2.0323139721 * 10**1,
                
                "c1": -7.8577991460 * 10**1,
                "c2": -7.0671612519 * 10**0,
                "c3": 2.7744056480 * 10**1,
                "c4": 7.5588609922 * 10**-1,
                "c5": 3.6591813315 * 10**1
                })

#Linear regression coefficients for the Design Draft model (Table 5 in paper Van Dorsser et al (2020))

Tdesign_coefs = dict({"intercept":0,
                     "c1": 1.7244153371,
                     "c2": 2.2767179246,
                     "c3": 1.3365379898,
                     "c4": -5.9459308905,
                     "c5": 6.2902305560*10**-2,
                     "c6": 7.7398861528*10**-5,
                     "c7": 9.0052384439*10**-3,
                     "c8": 2.8438560877
                     })

#Linear regression coefficients for the Empty Draft model (Table 4 in paper Van Dorsser et al (2020))

Tempty_coefs = dict({"intercept": 7.5740820927*10**-2,
                    "c1": 1.1615080992*10**-1,
                    "c2": 1.6865973494*10**-2,
                    "c3": -2.7490565381*10**-2,
                    "c4": -5.1501240744*10**-5,
                    "c5": 1.0257551153*10**-1, # Dummy DH Dry Bulk & DH Container
                    "c6": 2.4299435211*10**-1, # Dummy DH Tanker 
                    "c7": -2.1354295627*10**-1, # Dumb Barge
                    })

#Linear regresion coefficients for Design Capacity (DWT) model

capacity_coefs = dict({"intercept": -1.6687441313*10**1,
                     "c1": 9.7404521380*10**-1,
                     "c2": -1.1068568208,
                      })

### 2. Capacity index
Input:
- T_empty = Empty draft
- T_actual = Actual draft (maximum possible draft, taking ukc into account)

In [3]:
def cap_index(T_empty, T_actual):
    CI = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2) \
    + (CI_coefs["c3"] * T_actual) + (CI_coefs["c4"] * T_actual**2) \
    + (CI_coefs["c5"] * (T_empty * T_actual))
    
    return CI

### 3. Design draft

Linear regression model to estimate the design draft when this parameter is not known.
Input:

- vessel = String of the vessel type (Container, Dry, Barge, Tanker)
- L = Length of the vessel
- B = Beam of the vessel

In [4]:
def design_draft(vessel, L, B):
    
    if vessel == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
        
    elif vessel == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
        
    elif vessel == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
        
    elif vessel == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]
        
    elif vessel == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]
    
    T_design = Tdesign_coefs['intercept'] \
    + (dum_container * Tdesign_coefs['c1']) + (dum_dry * Tdesign_coefs['c2']) + (dum_barge * Tdesign_coefs['c3']) + (dum_tanker * Tdesign_coefs['c4']) \
    + (Tdesign_coefs['c5'] * dum_container * L**0.4 * B**0.6) + (Tdesign_coefs['c6'] * dum_dry * L**0.7 * B**2.6) \
    + (Tdesign_coefs['c7'] * dum_barge * L**0.3 * B**1.8) + (Tdesign_coefs['c8'] * dum_tanker * L**0.1 * B**0.3)
   
    return T_design

### 4. Empty draft

Linear regression model to estimate the empty draft when this parameter is not known.
Input:

- vessel = String of the vessel type (Container, Dry_SH, Dry_DH, Barge, Tanker)
- T_design = Design Draft
- L = Length of the vessel
- B = Beam of the vessel

In [5]:
def empty_draft(vessel, T_design, L, B):
    
    if vessel == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
        
    elif vessel == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,0]
    
    elif vessel == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
    
    elif vessel == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]
        
    elif vessel == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]
    
    T_empty = Tempty_coefs['intercept'] \
    + (Tempty_coefs['c1'] * B) + (Tempty_coefs['c2'] * ((L * T_design) / B)) + (Tempty_coefs['c3'] * (np.sqrt(L * B))) \
    + (Tempty_coefs['c4'] * (L * B * T_design)) + (Tempty_coefs['c5'] * dum_container) + (Tempty_coefs['c5'] * dum_dry) \
    + (Tempty_coefs['c6'] * dum_tanker) + (Tempty_coefs['c7'] * dum_barge)
    
    return T_empty

### 5. Design capacity

Linear regression model to estimate the maximum capacity (DWT) when this parameter is not known.
Input:

- L = Length of the vessel
- B = Beam of the vessel

In [6]:
def design_capacity(T_design, T_empty, L, B):
    
    DWT = capacity_coefs['intercept'] + (capacity_coefs['c1'] * L * B * T_design) \
    + (capacity_coefs['c2'] * L * B * T_empty)
    
    return DWT

### 6. Actual draft

The actual draft is the draft which is maximum possible taking ukc and the water depth into account.

- T_design = Design draft
- T_empty = Empty draft
- depth = minimum available water depth
- ukc - under keel clearance (depends on river bed, cargo type and shippers choice

In [7]:
def actual_draft(T_design, T_empty, depth, ukc):
    
    if (T_design <= depth-ukc):
        
        T_actual = T_design
        
    elif T_empty > (depth -ukc):
        
        T_actual =  (f"No trip possible. Available depth smaller than empty draft: {depth - T_empty} m")
    
    elif (T_design > depth):
        
        T_actual = depth - ukc
        
    
    
    return T_actual

### 7. Application

In [8]:
# Input variables

vessel = "Dry_SH"
L = 110
B = 11.4
depth1 = 1.5
depth2 = 3.6

print(f"The vesseltype is {vessel} with dimensions {L}x{B} m")

#Calculate the design draft

T_design = design_draft(vessel, L, B)
print (f"The design draft is: {round(T_design, 2)} m")

#Caluclate the empty draft

T_empty = empty_draft(vessel, T_design, L, B)
print (f"The empty draft is: {round(T_empty, 2)} m")

#Calculate the actual draft

T_actual_1 = actual_draft(T_design, T_empty, depth=depth1, ukc=0.1)
T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.1)

print (f"The actual draft for depth1 is: {round(T_actual_1, 2)} m")
print (f"The actual draft for depth2 is: {round(T_actual_2, 2)} m")

#Calculate the capacity index

CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)

print (f"The capacity index for depth1 is: {round(CI_1, 2)}")
print (f"The capacity index for depth2 is: {round(CI_2, 2)}")

#Calculate the design capacity

DWT_dc = design_capacity(T_design, T_empty, L, B)
#print (f"The design capacity is: {round(DWT_dc, 0)} t")

# Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type

mod_capacity=DWT_dc*CI_1/CI_2

print(f"The capacity for minimum operational draft is: {round(mod_capacity,0)} t")

#Define the payload at design conditions by subtracting an assumed percentage of the DWT for consumables: default = 6%
consumable_design=0.06
consumable_lowwater=0.04

payload_design = (1-consumable_design)*DWT_dc
payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc

print (f"The design capacity is: {round(DWT_dc, 0)} t")
print(f"The payload for design draft is: {round(payload_design)} t")
print(f"The capacity for minimum operational draft is: {round(mod_capacity,0)} t")
print(f"The payload for minimum operational draft is: {round(payload_lowwater)} t")

The vesseltype is Dry_SH with dimensions 110x11.4 m
The design draft is: 3.44 m
The empty draft is: 0.76 m
The actual draft for depth1 is: 1.4 m
The actual draft for depth2 is: 3.44 m
The capacity index for depth1 is: 35.63
The capacity index for depth2 is: 156.72
The capacity for minimum operational draft is: 710.0 t
The design capacity is: 3125.0 t
The payload for design draft is: 2937 t
The capacity for minimum operational draft is: 710.0 t
The payload for minimum operational draft is: 585 t


In [9]:
#Application of the capacity index to calculate the load factor

RCI = CI_2
print (f"The relative capacity index is: {round(RCI,2)}")

#Baseline capacity

BC = (DWT / RCI) * 100
print (f"The baseline capacity is: {round(BC, 2)} t")

#The absolute capacity at 1.6m (depth1):

AC = (BC * (CI_1 / 100))
print (f"The absolyte capacity is {round(AC, 2)} t")

#The load facor

lf = (AC / DWT)
print (f"The load factor at {depth1}m is: {round(lf, 2)}")

The relative capacity index is: 156.72


NameError: name 'DWT' is not defined

### 8. Comparison with linear interpolation 

In [ ]:
vessel = "Dry"
L = 135
B = 11.40
depth1 = 2.4
depth2 = 3.7

T_design = 3.00 #Input variable in OpenCLSim model
T_empty = 1.00 #Input variable in OpenCLSim model


T_actual_1 = actual_draft(T_design, T_empty, depth1, ukc=0.25)
T_actual_2 = actual_draft(T_design, T_empty, depth2, ukc=0.25)

CI_1 = cap_index(T_empty, T_actual=T_actual_1)
CI_2 = cap_index(T_empty, T_actual=T_actual_2)

DWT = 3900 #Input variable in OpenCLSim model

#Baseline capacity
BC = DWT/RCI * 100

#Absolute capacity at 2.4m (depth1)
AC = (BC * (CI_1 / 100))

#Load factor

lf = (AC / (DWT))
print (f"Load facor: {round(lf, 2)}")

# CapacityIndex Class

In [ ]:
Length = 110
Beam = 11.45

class CapacityIndex():
    
    def __init__(self, Length, Beam, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Length = Length
        self.Beam = Beam
        
        
    def design_draft(self, vessel):
        
        
        Tdesign_coefs = dict({"intercept":0,
                         "c1": 1.7244153371,
                         "c2": 2.2767179246,
                         "c3": 1.3365379898,
                         "c4": -5.9459308905,
                         "c5": 6.2902305560*10**-2,
                         "c6": 7.7398861528*10**-5,
                         "c7": 9.0052384439*10**-3,
                         "c8": 2.8438560877
                         })

    
        if vessel == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]

        elif vessel == "Dry":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]

        elif vessel == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vessel == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        T_design = Tdesign_coefs['intercept'] \
        + (dum_container * Tdesign_coefs['c1']) + (dum_dry * Tdesign_coefs['c2']) + (dum_barge * Tdesign_coefs['c3']) + (dum_tanker * Tdesign_coefs['c4']) \
        + (Tdesign_coefs['c5'] * dum_container * Length**0.4 * B**0.6) + (Tdesign_coefs['c6'] * dum_dry * Length**0.7 * B**2.6) \
        + (Tdesign_coefs['c7'] * dum_barge * Length**0.3 * B**1.8) + (Tdesign_coefs['c8'] * dum_tanker * Length**0.1 * B**0.3)
        
        return T_design
    
    def empty_draft(self, vessel, T_design):
    
        if vessel == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]

        elif vessel == "Dry":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]

        elif vessel == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vessel == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        T_empty = Tempty_coefs['intercept'] \
        + (Tempty_coefs['c1'] * Beam) + (Tempty_coefs['c2'] * ((Length * T_design) / Beam)) + (Tempty_coefs['c3'] * (np.sqrt(Length * Beam))) \
        + (Tempty_coefs['c4'] * (Length * Beam * T_design)) + (Tempty_coefs['c5'] * dum_container) + (Tempty_coefs['c5'] * dum_dry) \
        + (Tempty_coefs['c6'] * dum_tanker) + (Tempty_coefs['c7'] * dum_barge)

        return T_empty
    
    def actual_draft(self, depth, ukc):
    
        if (T_design <= depth):

            T_actual = T_design

        elif T_empty > (depth -ukc):

            T_actual =  (f"No trip possible. Available depth smaller than empty draft: {depth - T_empty} m")

        elif (T_design > depth):

            T_actual = depth - ukc

        return T_actual
    
    def cap_index(self, T_empty, T_actual):
        
        CI = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2) \
        + (CI_coefs["c3"] * T_actual) + (CI_coefs["c4"] * T_actual**2) \
        + (CI_coefs["c5"] * (T_empty * T_actual))

        return CI
    
    def design_capacity(self, T_design, T_empty):
    
        DWT = capacity_coefs['intercept'] + (capacity_coefs['c1'] * Length * Beam * T_design) \
        + (capacity_coefs['c2'] * Length * Beam * T_empty)

        return DWT

In [ ]:
Va = CapacityIndex(110, 11.45)

T_design = Va.design_draft("Container")
T_empty = Va.empty_draft("Container", T_design=T_design)
T_actual_1 = Va.actual_draft(depth=1.6, ukc=0.2)
T_actual_2 = Va.actual_draft(depth=3.7, ukc=0.2)

Capindex_1 = Va.cap_index(T_empty=T_empty, T_actual=T_actual_1)
Capindex_2 = Va.cap_index(T_empty=T_empty, T_actual=T_actual_2)
DWT = Va.design_capacity(T_design, T_empty)

In [ ]:
#Baseline cap
BC = (DWT / Capindex_2) * 100
AC = BC * (Capindex_1/100)
AC
lf = AC/DWT
lf